## Imports

In [ ]:
try:
    import pylibjpeg
except:
    # Offline dependencies:
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !cp ../input/rsna-2022-whl/efficientnet_v2_s-dd5fe13b.pth  /root/.cache/torch/hub/checkpoints/

    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
    !pip install /kaggle/input/rsna-2022-whl/{torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl,torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl}

In [3]:
import glob
import os
import re
import sys
import cv2
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pydicom
import torch

from torchvision.models.feature_extraction import create_feature_extractor
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torch import nn
import albumentations

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master/')
sys.path.append('../input/pretrained-models-pytorch/pretrained-models.pytorch-master')
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')
from efficientnet_pytorch import EfficientNet
from pretrainedmodels import se_resnext101_32x4d
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects

RSNA_2022_PATH = '../input/rsna-2022-cervical-spine-fracture-detection'
TRAIN_IMAGES_PATH = f'{RSNA_2022_PATH}/train_images'
TEST_IMAGES_PATH = f'{RSNA_2022_PATH}/test_images'
CONVNEXT_CHECKPOINTS_PATH = '../input/rsna-models/convnext_384'
IMAGE_SIZE = 512
MODEL_NAMES = [f'convnext-f{i}' for i in range(2,3)]
FRAC_COLS = [f'C{i}_frac' for i in range(1, 8)]
VERT_COLS = [f'C{i}_vert' for i in range(1, 8)]

try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except:
    IS_KAGGLE = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_SIZE = 32


## Prepare Data

In [4]:
def load_df_test():
    df_test = pd.read_csv(f'{RSNA_2022_PATH}/test.csv')

    if df_test.iloc[0].row_id == '1.2.826.0.1.3680043.10197_C1':
        df_test = pd.DataFrame({
            "row_id": ['1.2.826.0.1.3680043.22327_C1', '1.2.826.0.1.3680043.25399_C1', '1.2.826.0.1.3680043.5876_C1'],
            "StudyInstanceUID": ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876'],
            "prediction_type": ["C1", "C1", "patient_overall"]}
        )
    return df_test

df_test = load_df_test()
df_test

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.25399_C1,1.2.826.0.1.3680043.25399,C1
2,1.2.826.0.1.3680043.5876_C1,1.2.826.0.1.3680043.5876,patient_overall


In [5]:
test_slices = glob.glob(f'{TEST_IMAGES_PATH}/*/*')
test_slices = {re.findall(f'{TEST_IMAGES_PATH}/(.*)/(.*).dcm', s)[0] for s in test_slices}
df_test_slices = pd.DataFrame(data=test_slices, columns=['StudyInstanceUID', 'Slice']).astype({'Slice': int}).sort_values(['StudyInstanceUID', 'Slice']).reset_index(drop=True)


## Bounding Box Prediction

In [7]:
test_transform = albumentations.Compose([
        albumentations.Normalize(mean=(0.456, 0.456, 0.456), std=(0.224, 0.224, 0.224), max_pixel_value=255.0, p=1.0)
])

In [11]:
def bbox_window(x, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    x = np.clip(x, lower, upper)
    x = x - np.min(x)
    x = x / np.max(x)
    return x

class BboxDataset(torch.utils.data.Dataset):    
    def __init__(self, series_list):        
        self.series_list = series_list    
    def __len__(self):        
        return len(self.series_list)    
    def __getitem__(self,index):        
        return index

class BboxCollator(object):
    def __init__(self, series_list, df):
        self.df = df
        self.series_list = series_list
        
    def _load_dicom_array(self, f):
        dicom_files = glob.glob(os.path.join(f, '*.dcm'))
        dicoms = [pydicom.dcmread(d) for d in dicom_files]
        
        M = np.float32(dicoms[0].RescaleSlope)
        B = np.float32(dicoms[0].RescaleIntercept)
        
        z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
        sorted_idx = np.argsort(z_pos)
        dicom_files = np.asarray(dicom_files)[sorted_idx]
        dicoms = np.asarray(dicoms)[sorted_idx]
        
        selected_idx = [int(0.3*len(dicom_files)), int(0.4*len(dicom_files)), int(0.5*len(dicom_files)), int(0.6*len(dicom_files))]
        selected_dicom_files = dicom_files[selected_idx]
        selected_dicoms = dicoms[selected_idx]
        
        dicoms = np.asarray([d.pixel_array.astype(np.float32) for d in selected_dicoms])
        dicoms = dicoms * M # RescaleSlope
        dicoms = dicoms + B # RescaleIntercept
        dicoms = bbox_window(dicoms, WL=400, WW=650)
        
        th = threshold_otsu(dicoms)
        
        dicoms = dicoms >=th
        dicoms = remove_small_objects(dicoms, 10)
        return dicoms, dicom_files, selected_dicom_files
    
    def __call__(self, batch_idx):
        series_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images/'+self.df['StudyInstanceUID'].values[batch_idx] + "/"
        dicoms, dicom_files, selected_dicom_files = self._load_dicom_array(series_dir[0])
        
        image_list = []
        for i in range(len(dicom_files)):
            name = self.df['StudyInstanceUID'].values[batch_idx]
            image_list.append(name[0])
        selected_image_list = []
        
        for i in range(len(selected_dicom_files)):
            name = self.df['StudyInstanceUID'].values[batch_idx]
            selected_image_list.append(name[0])
            
        x = np.zeros((4, 3, dicoms.shape[1], dicoms.shape[2]), dtype=np.float32)
        
        for i in range(4):
            x[i,0] = dicoms[i]
            x[i,1] = dicoms[i]
            x[i,2] = dicoms[i]
        return torch.from_numpy(x), image_list, selected_image_list, self.series_list[batch_idx[0]]

In [12]:
class efficientnet(nn.Module):
    def __init__(self ):
        super().__init__()
        self.net = EfficientNet.from_name('efficientnet-b0')
        in_features = self.net._fc.in_features
        self.last_linear = nn.Linear(in_features, 4)
    def forward(self, x):
        x = self.net.extract_features(x)
        x = self.net._avg_pooling(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

In [ ]:
bbox_image_id_list = df_test['StudyInstanceUID'].values
model = efficientnet()
print(model.load_state_dict(torch.load('../input/rsna-models/epoch29')))
model = model.cuda()
model.eval()

pred_bbox = np.zeros((len(bbox_image_id_list)*4,4),dtype=np.float32)
bbox_csv= pd.DataFrame()

selected_image_list_valid = {}
datagen = BboxDataset(series_list=bbox_image_id_list)
collate_fn = BboxCollator(series_list=bbox_image_id_list, df=df_test)
generator = DataLoader(dataset=datagen, collate_fn=collate_fn, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)
total_steps = len(generator)


# Predict Bounding Boxes
for i, (images, image_list, selected_image_list, series_id) in tqdm(enumerate(generator), total=total_steps):
    with torch.no_grad():
        start = i*4
        end = start+4
        if i == len(generator)-1:
            end = len(generator.dataset)*4
        images = images.cuda()
        res = images.shape[2]
        logits = model(images)
        bbox = np.squeeze(logits.cpu().data.numpy())
        pred_bbox[start:end] = bbox
        
        xmin = np.round(min([bbox[0,0], bbox[1,0], bbox[2,0], bbox[3,0]])*res)
        ymin = np.round(min([bbox[0,1], bbox[1,1], bbox[2,1], bbox[3,1]])*res)
        xmax = np.round(max([bbox[0,2], bbox[1,2], bbox[2,2], bbox[3,2]])*res)
        ymax = np.round(max([bbox[0,3], bbox[1,3], bbox[2,3], bbox[3,3]])*res)
        selected_image_list_valid[str(series_id)] = {'xmin':max(0, xmin/res), 'ymin' : max(0, ymin/res), 'xmax' : min(1.0, xmax/res), 'ymax' : min(1.0, ymax/res)}

## Image Level Prediction

In [15]:
class seresnext101(nn.Module):
    def __init__(self ):
        super().__init__()
        self.net = se_resnext101_32x4d(num_classes=1000, pretrained=None)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        in_features = self.net.last_linear.in_features
        self.last_linear = nn.Linear(in_features, 7)
        self.nn_fracture = torch.nn.Sequential(
            torch.nn.Linear(2048, 7),
        )
        self.nn_vertebrae = torch.nn.Sequential(
            torch.nn.Linear(2048, 7),
        )
    def forward(self, x):
        x = self.net.features(x)
        x = self.avg_pool(x)
        feature = x.view(x.size(0), -1)
        return self.nn_fracture(feature), self.nn_vertebrae(feature)
    
    def predict(self, x):
        frac, vert = self.forward(x)
        return torch.sigmoid(frac), torch.sigmoid(vert)

In [16]:
def load_model(model, name, path='.'):
    data = torch.load(os.path.join(path, f'{name}.tph'), map_location=DEVICE)
    print(model.load_state_dict(data))
    return model

In [ ]:
def window(name, img, WL=950, WW=1900):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    
    return X

class RSNADataset(torch.utils.data.Dataset):
    def __init__(self, df, bbox_dict, image_size,  transform):
        self.df = df
        self.bbox_dict=bbox_dict
        self.image_size=image_size
        self.transform=transform
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        
        study_id = self.df['StudyInstanceUID'][index]
        series_id = self.df['Slice'].values[index]
        
        data1 = pydicom.dcmread('../input/rsna-2022-cervical-spine-fracture-detection/test_images/'+ str(study_id)+ f'/{int(series_id)}' + '.dcm')
    
        x1 = data1.pixel_array
        res = x1.shape[1]
        x1c = x1*data1.RescaleSlope+data1.RescaleIntercept

        # Windowing values are referenced in this document https://arxiv.org/abs/2010.13336
        x1 = np.expand_dims(window(study_id, x1c, WL=500, WW=1800), axis=2)
        x2 = np.expand_dims(window(study_id, x1c, WL=400, WW=650), axis=2)
        x3 = np.expand_dims(window(study_id, x1c, WL=80, WW=300), axis=2)
        
        # threshold_otsu, remove_small_objects for 'Bone' channel
        th = threshold_otsu(x2) 
        x2 = x2>=th
        x2 = remove_small_objects(x2, 10)
        x = np.concatenate([x1, x2, x3], axis=2)

        bbox = self.bbox_dict[study_id]
        
        # Predicted Bounding Boxes are expanded by 20%
        x = x[int(max(0, bbox['ymin']*res*0.8)):int(min(res, bbox['ymax']*res*1.2)),int(max(0, bbox['xmin']*res*0.8)):int(min(res, bbox['xmax']*res*1.2)),:]        

        x = cv2.resize(x, (self.image_size,self.image_size))
        x = self.transform(image=x)['image']
        x = x.transpose(2, 0, 1)
        
        # Model has two heads, predicting position of fracture and vertebrae
        if 'C1_fracture' in self.df:
            frac_targets = torch.as_tensor(self.df.iloc[index][['C1_fracture', 'C2_fracture', 'C3_fracture', 'C4_fracture',
                                                            'C5_fracture', 'C6_fracture', 'C7_fracture']].astype('float32').values)
            vert_targets = torch.as_tensor(self.df.iloc[index][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].astype('float32').values)
            
            # only enable targets are visible on the current slice
            frac_targets = frac_targets * vert_targets  
            return x, frac_targets, vert_targets
        return x

## Inference

In [18]:
ds_test = RSNADataset(df_test_slices, selected_image_list_valid, IMAGE_SIZE, test_transform)
models = []
model = seresnext101()
model.load_state_dict(torch.load('../input/rsna-models/seresnext1010_22234full_otsu.tph', map_location = DEVICE))
models.append(model.cuda())

In [19]:
from typing import List

def predict_net(models, ds):
    dl_test = torch.utils.data.DataLoader(ds, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False, 
                                          num_workers=2, 
                                          pin_memory = True)
    
    for m in models:
        m.eval()

    with torch.no_grad():
        predictions = []
        for idx, X in enumerate(dl_test):
            pred = torch.zeros(len(X), 14).to(DEVICE)
            for m in models:
                y1, y2 = m.predict(X.float().to(DEVICE)) # self.nn_fracture(feature), self.nn_vertebrae(feature)
                pred += torch.concat([y1, y2], dim=1) / len(models)
            predictions.append(pred)
        return torch.concat(predictions).cpu().numpy()

In [20]:
predicted = predict_net(models, ds_test)
predicted_df = pd.DataFrame(
    data=predicted, columns=[f'C{i}_frac' for i in range(1, 8)] + [f'C{i}_vert' for i in range(1, 8)]
)

In [ ]:
df_test_pred = pd.concat([df_test_slices, predicted_df], axis=1).sort_values(['StudyInstanceUID', 'Slice'])
df_test_pred

In [22]:
# Weighted averaging of each patient to get the probability of fracture position in each vertebrae

def patient_prediction(df):
    c1c7 = np.average(df[FRAC_COLS].values, axis=0, weights=df[VERT_COLS].values)
    pred_patient_overall = 1 - np.prod(1 - c1c7)
    return pd.Series(data=np.concatenate([[pred_patient_overall], c1c7]), index=['patient_overall'] + [f'C{i}' for i in range(1, 8)])

df_patient_pred = df_test_pred.groupby('StudyInstanceUID').apply(lambda df: patient_prediction(df))
df_patient_pred

,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,
1.2.826.0.1.3680043.22327,0.349046,0.058591,0.069128,0.022744,0.014510,0.013225,0.051244,0.176148
1.2.826.0.1.3680043.25399,0.239549,0.035188,0.002994,0.018086,0.016377,0.007316,0.106921,0.076731
1.2.826.0.1.3680043.5876,0.344666,0.047372,0.217216,0.007969,0.005617,0.008373,0.018365,0.084791


## Submissions

In [23]:
df_sub = df_test.copy()
df_sub = df_sub.set_index('StudyInstanceUID').join(df_patient_pred)
df_sub['fractured'] = df_sub.apply(lambda r: r[r.prediction_type], axis=1)
df_sub

,row_id,prediction_type,patient_overall,C1,C2,C3,C4,C5,C6,C7,fractured
StudyInstanceUID,,,,,,,,,,,
1.2.826.0.1.3680043.22327,1.2.826.0.1.3680043.22327_C1,C1,0.349046,0.058591,0.069128,0.022744,0.014510,0.013225,0.051244,0.176148,0.058591
1.2.826.0.1.3680043.25399,1.2.826.0.1.3680043.25399_C1,C1,0.239549,0.035188,0.002994,0.018086,0.016377,0.007316,0.106921,0.076731,0.035188
1.2.826.0.1.3680043.5876,1.2.826.0.1.3680043.5876_C1,patient_overall,0.344666,0.047372,0.217216,0.007969,0.005617,0.008373,0.018365,0.084791,0.344666


In [24]:
df_sub[['row_id', 'fractured']].to_csv('submission.csv', index=False)